In [4]:
!pip install unidecode
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
from unidecode import unidecode

def csv_files(path):
    """Função usada para ler os ficheiros que tem o simbolo do Euro"""
    import csv
    f = open(path, encoding='windows-1252', errors='ignore')
    data = []
    for row in csv.reader(f, delimiter=','):
        data.append(row)
    col = [*data[0]]
    data.pop(0)
    df = pd.DataFrame(data, columns=col)
    return df, col

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist

def change_incorret(df, df_index, col_chg, incorrect_value, new_value):
    list_sites = list(df[df[col_chg]==incorrect_value][tw_index])
    df.loc[df[df_index].isin(list_sites), col_chg] = new_value
    #print(df.loc[df[df_index].isin(list_sites)][[df_index, col_chg]])
       

# Reading and Fit Excel File


In [ ]:
# Read CZ towerdb files
def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index, cols_number, col_dates, format='normal', type_date="%d/%m/%Y"):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ']
        #lista = []

        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(0)
                else:
                    lista.append(index)
            df[column] = lista

        return df

    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) and not isinstance(date_obj, str) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    df = pd.read_excel(path, sheet_name = sheetname, engine='openpyxl', skiprows = n_skiprows)
    df = df.iloc[:,n_skip_columns:]
    date_parser(df, col_dates, format, type_date)
    # define the entiry columns which doesn't have NaN 
    df = df.dropna(subset=[site_index], axis=0)
    df = replace_values(df, cols_number)

    for col in cols_number:
        df[col] = df[col].astype(int).apply(lambda x: f'{x:,}')
    
    df = df.fillna('')

    return df

path_towerdb = '/content/TowerDB_FY21-06 June-Skylon VF CZ_v1.xlsx'
towerdb_col = lower_str(["Code (Duplicate)","Site Status","VF - In scope / out of scope (Generalised scoping)","Site in Skylon scope Actual (From Site List Sheet )",\
               "Legacy Site Code(Duplicate)","TIMS Site Code","Legacy Site Code","Site Name","Macro Region","Province","Municipality","Inhabitants","Address",\
               "Ground Register","Altitude","Latitude","Longitude","Categorization by Inhabitants","Categorization by Transmission Sys",\
               "Categorization by Site Type","Categorization by Transmission Sys (subcluster)","Other internal Categorization 1 (Identify ACQ Sites)",\
               "Other internal Categorization 2 Energy provider (Eon/ LL)","DAS+Macro","DAS (Yes/ No)","DAS Ownership (Complete/ Partial/ 3rd Party)",\
               "Active/ Passive DAS","# of remote units/ radiating points","Type of Structure","Distance highest antenna to ground level","GBT Tower height",\
               "POD ID","Energy Consumption LTM (kwh)","Annual Energy cost LTM (Euros)","Infrastructure ready (existing)/ to be ready (new)",\
               "Infrastructure to be dismantled by","Radio equipments to be deactivated by","Infrastructure to be shared by","Technology VOD","Fibre / Microwave",\
               "Vertical passive structure owner","Room configuration (detailed)","Shelter passive structure ownership","Type of Air Conditioning",\
               "Number of cabinets (Full Capacity)","Number of Antenna (Full Capacity)","Number of MW (Full Capacity)","Counterpart","# of Lease Contracts",\
               "Current annual lease fees ","Current other fees (Maintenance)","Current other fees","(Average) residual duration - Lease contract",\
               "(Average) residual duration - Maintenance contract","(Average) residual duration - Lease & Maintenance both","# of Tenants Agreements",\
               "Current Total Annual Hosting Fees","Tenant (name/ID) MNO1 (Česká telekomunikační infrastruktura a.s.)","Annual Fee per Tenant MNO1",\
               "Annual Energy Fee MNO1","Annual Maintenance Fee MNO1","Other Services Fee MNO1","Residual duration MNO1 (Years)",\
               "Tenant (name/ID) MNO2 (T-Mobile Czech Republic a.s.)","Annual Fee per Tenant MNO2","Annual Energy Fee MNO2","Annual Maintenance Fee MNO2",\
               "Other Services Fee MNO2","Residual duration MNO2 (days)","Tenant (name/ID) MNO3","Annual Fee per Tenant MNO3","Annual Energy Fee MNO3",\
               "Annual Maintenance Fee MNO3","Other Services Fee MNO3","Residual duration MNO3","# of OTMOs","Annual Fee from OTMOs","Annual Energy Fee from OTMOs",\
               "Annual Maintenance Fee OTMOs","Other Services Fee OTMOs","Average residual duration (days)","Check","Strategic Macro Sites","Critical Sites",\
               "Case A Core Site","Macro Site - Transmission Hub Site","Macro Site - Transmission Hub Site with/without Shelters","Transmission Sites",\
               "Room Configuration","Power Supply","Air Conditioning","Active Sharing Arrangements involving the Operator","VF-CZ Demerger phase",\
               "EVO Location [FAR Site ID] ","Billing Trigger date ","Strategic_Site_Bucket","Critical Site Bucket","Subsequent_Sharing_Arrangement",\
               "First_Active_Sharing_Deployment_Type","First_Active_Sharing_Start_Date","First_Active_Sharing_End_Date","Decommissioned_Site","Wip_Site",\
               "Bts_Site","Transfer_Date_Of_Consent_Required_Sites","Sites_As_Metered_Estimated","Date_Of_Equipment_Removal"])

dates = ['Billing Trigger date ', "Infrastructure ready (existing)/ to be ready (new)", "Infrastructure to be dismantled by",\
         'First_Active_Sharing_Start_Date','First_Active_Sharing_End_Date', 'Transfer_Date_Of_Consent_Required_Sites',\
         'Date_Of_Equipment_Removal']
num_parse = ["Inhabitants", "Current annual lease fees ","(Average) residual duration - Lease contract",\
             "(Average) residual duration - Maintenance contract","(Average) residual duration - Lease & Maintenance both",\
             "# of Tenants Agreements","Current Total Annual Hosting Fees", "Annual Fee per Tenant MNO1",\
             "Annual Energy Fee MNO1", "Annual Fee per Tenant MNO2","Annual Energy Fee MNO2",\
             "Residual duration MNO2 (days)", "Annual Fee from OTMOs","Annual Energy Fee from OTMOs",\
             "Other Services Fee OTMOs","Average residual duration (days)"]
tab = 'Tower DB'
row = 3
towerdb = read_files(path_towerdb,tab, 3, 0,"Code (Duplicate)",num_parse,dates)
towerdb.columns = lower_str(list(towerdb.columns))
towerdb = towerdb[towerdb_col]

path_msa = '/content/TowerDB_CzechRepublic_20210731 (3).csv'
msa = pd.read_csv(path_msa,encoding='ISO-8859-1').fillna('')
msa.columns = lower_str(list(msa.columns))
msa = msa.rename(columns={'ï»¿code (duplicate)': 'code (duplicate)',
                          'tenant (name/id) mno1 (ä\x8ceskã¡ telekomunikaä\x8dnã\xad infrastruktura a.s.)': 'tenant (name/id) mno1 (česká telekomunikační infrastruktura a.s.)'})
msa_cols = list(msa.columns)

def check_columns_received(df, bill_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in bill_cols if i not in twdb_col]
    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

"""Check columns received"""
df_cols = check_columns_received(towerdb, msa_cols)
df_cols
#No colum

In [ ]:
def change_incorret(df, df_index, col_chg, incorrect_value, new_value):
    list_sites = list(df[df[col_chg]==incorrect_value][tw_index])
    df.loc[df[df_index].isin(list_sites), col_chg] = new_value
    print(df.loc[df[df_index].isin(list_sites)][[df_index, col_chg]])

change_incorret(towerdb, tw_index, 'subsequent_sharing_arrangement', 'NO', 'No')

In [ ]:
towerdb.to_excel('/content/TowerDB_CzechRepublic_20210831.xlsx', index=False)

In [ ]:
towerdb.to_csv('/content/TowerDB_CzechRepublic_20210831.csv', index=False)

In [15]:
"""Defining variables which is gonna be reusable in checks"""
tw_index = "code (duplicate)"
tw_doer = "date_of_equipment_removal"
tw_status = "site status"
tw_bts = 'bts_site'
tw_bill = "billing trigger date "
tw_wip = 'wip_site'
tw_decom = 'decommissioned_site'
tw_critical = 'critical sites'

Ver essa necessidade de unidecode

In [ ]:
path_msa = '/content/TowerDB_CzechRepublic_20210731 (3).csv'
msa = pd.read_csv(path_msa,encoding='ISO-8859-1').fillna('')
cols_con = ['Province','Municipality','Address']
for col in cols_con:
   msa[col] = msa[col].astype(str).apply(unidecode)
msa = msa.applymap(lambda x: x.encode('unicode_escape').decode('ISO-8859-1') if isinstance(x, str) else x)
msa.columns = lower_str(towerdb_col)
msa.head(1)
#msa.columns = msa_cols


In [ ]:
# Check Dates Format (dd-mm-YYYY)
"""Não deve haver NAN ou NAT na coluna, fazer fillna('') antes"""
def check_date_columns(df, df_index,status_col,bts_col, wip_col, columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)

    filtered = df[[df_index, status_col,bts_col, wip_col]]

    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                            
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col,bts_col, wip_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')

dates_cols = [tw_index,'Billing Trigger date ', 'First_Active_Sharing_Start_Date',\
              'First_Active_Sharing_End_Date', 'Transfer_Date_Of_Consent_Required_Sites',\
              'Date_Of_Equipment_Removal'] 
dates_cols = lower_str(dates_cols)

dates = ['Billing Trigger date ', 'First_Active_Sharing_Start_Date',\
              'First_Active_Sharing_End_Date', 'Transfer_Date_Of_Consent_Required_Sites',\
              'Date_Of_Equipment_Removal']           
dates = lower_str(dates)

#date_parser(towerdb, lower_str(dates), 2, 1, 'mixed')
#tw_in_service = tw_in_service[dates_cols]
tw_in_service = towerdb[towerdb['site status']=='In Service']
df_dates_errors = check_date_columns(tw_in_service,tw_index, tw_status, tw_bts, tw_wip, lower_str(dates_cols), 2)
df_dates_errors
#Errors em varias colunas

Validaton for general sites picklist

In [ ]:
def check_picklist_v1(df,df_index,df_status, df_cols, picklist_dict):
    from collections import defaultdict

    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)
    """for id in df_picklist[df_index]:
        if id not in new_dic.keys():
            new_dic[df_index].append(id)""" 
            
    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]): 
                if pd.isnull(value) or pd.isna(value) or value=='' or value=='nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors[df_index] = df_picklist[df_index]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, df_status]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
    #df_errors = df_errors.reset_index()
    return df_errors

dic_gen = ['code (duplicate)', 'site status', 'categorization by transmission sys']
pick_gen = {
    'categorization by transmission sys': ['Macro','Public DAS','Repeater','Transmission']
    }
df_pick_general_errors = check_picklist_v1(towerdb, tw_index, tw_status, dic_gen, pick_gen)
df_pick_general_errors
#Pipeline sites without categorization

Validation for On air sites

In [ ]:
picklist_cz = {
    'categorization by transmission sys': ['Macro','Public DAS','Repeater','Transmission'],
    'categorization by site type':['DAS passive','GBT','RTT'],
    'strategic macro sites':['Yes','No'],
    'critical sites':['Yes','No'],
    'case a core site':['Non Core', 'Case A','Case B'],
    'macro site - transmission hub site':['Yes','No'],
    'macro site - transmission hub site with/without shelters': ['With shelters','Without shelters','Non Transmission Hub Site'],
    'transmission sites': ['No','Yes'],
    'room configuration': ['Indoor','Outdoor',"Indoor & Outdoor"],
    'power supply':['AC','DC','AC & DC','No Power'],
    'air conditioning': ['No','Yes; Indoor Air Conditioning','Yes; Indoor Free Air cooling / Free cooling units'],
    'active sharing arrangements involving the operator':['MORAN (On VF equipment)','MORAN (On non-VF equipment)','MOCN with Spectrum Pooling','Partial Active-Passive','No Active Sharing'],
    'vf-cz demerger phase': ['Cancelled','Phase 1','Phase 2','Skylon BTS'],
    'strategic_site_bucket': ['Yes - 0-5%','Yes - 5-10%','Non Strategic'],
    'critical site bucket': ['Beyond 10%','Within 10%','Non Critical'],
    'subsequent_sharing_arrangement': ['Yes','No'],
    'decommissioned_site': ['Yes','No'],
    'wip_site': ['Yes','No'],
    'bts_site': ['Yes','No'],
    'sites_as_metered_estimated': ['Estimated Model','Metered Model']
}
dic_cols = [tw_index,'categorization by transmission sys',\
            'categorization by site type',\
            'strategic macro sites',\
            'critical sites',\
            'case a core site',\
            'macro site - transmission hub site',\
            'macro site - transmission hub site with/without shelters',\
            'transmission sites',\
            'room configuration',\
            'power supply',\
            'air conditioning',\
            'active sharing arrangements involving the operator',\
            'vf-cz demerger phase',\
            'strategic_site_bucket',\
            'critical site bucket',\
            'subsequent_sharing_arrangement',\
            'decommissioned_site',\
            'wip_site',\
            'bts_site',\
            'sites_as_metered_estimated']
df_pick_actives_errors = check_picklist_v1(tw_in_service, tw_index, tw_status, dic_cols, picklist_cz)
df_pick_actives_errors
#Erros em varias colunas

Month-on-Month BTS sites

In [38]:
def check_mom_bts(df_tw, tw_index,status_col, tw_col, df_msa, msa_index, msa_col):

    msa_bts = [i for i in df_msa[df_msa[msa_col]=='Yes'][msa_index]]

    tw_bts = [i for i in df_tw[df_tw[tw_col]=='Yes'][tw_index]]

    out_tower_bts = [i for i in msa_bts if i not in tw_bts]
    filtered = df_tw.loc[df_tw[tw_index].isin(out_tower_bts)]

    return filtered[[tw_index,status_col, tw_col]]    
        
tw_filtered = towerdb[towerdb['site status']=='In Service']

df_mom_bts = check_mom_bts(tw_filtered, tw_index, tw_status, tw_bts, msa, tw_index, tw_bts)
df_mom_bts

#df_mom_decom = check_mom_bts(tw_filtered, tw_index,tw_status, tw_decom, msa, tw_index, tw_decom)
#df_mom_decom
# No errors in Both

,code (duplicate),site status,decommissioned_site


In [56]:
#Read UIP File
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', 'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip = pd.read_excel('/content/UserInput_CzechRepublic_20210831.xlsx',sheet_name='SiteLevel',usecols=[0,2,3],skiprows=3).fillna('')
uip.columns = uip_names

msa_sites = [i for i in msa[tw_index]]
tw_sites = [str(i) for i in towerdb[tw_index]]
uip_sites = [str(i) for i in uip['Site_ID']]


Verificar Errors de Sites Numericos com String
Alguns sites estão com Integer ou String no ficheiro anterior

In [ ]:
def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    filtered = df_towerdb[[tw_index, status_col]]

    #Finding for ALL NEW SITES
    new_site = [i for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    new_sites['New_Sites'] =  new_sites['New_Sites'].astype(int)
    new_sites = pd.merge(new_sites, filtered, how='left', left_on=['New_Sites'], right_on=tw_index)
    
    new_sites = new_sites[['New_Sites', status_col]]

    
    #list of new sites out of UIP sheet
    bts_out_uis = [i for i in df_towerdb[df_towerdb[bts_col]=='Yes'][tw_index] if i not in uip_list]
    bts_out_uis = pd.DataFrame(bts_out_uis, columns=['Bts_Sites_Out_UIS_File'])
    bts_out_uis = pd.merge(bts_out_uis, filtered, how='left', left_on=['Bts_Sites_Out_UIS_File'], right_on=tw_index)
    bts_out_uis = bts_out_uis[['Bts_Sites_Out_UIS_File', status_col]]

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = df[bill_col].astype('datetime64[s]')
    #print(current_date)
    df_site_bts = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]
    

    return new_sites, bts_out_uis, df_site_bts[[tw_index,status_col, bts_col, bill_col]]

new_sites, bts_sites_out_uis, df_bts_demerged = check_new_sites(towerdb, tw_index, tw_bts, tw_bill, tw_status, msa_sites, tw_sites, uip_sites)

print(new_sites)
print('\n')
print(bts_sites_out_uis)
print('\n')
print(df_bts_demerged)
print('\n')

In [ ]:
def check_wip(df_tw,tw_index, wip_tw, tw_bts, df_msa, msa_index, msa_wip):

    wip_msa = [i for i in df_msa[df_msa[msa_wip]=='Yes'][msa_index]]
    
    tw_wip_sites = [str(i) for i in df_tw[df_tw[wip_tw]=='Yes'][tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, wip_tw, tw_bts]]
    
    wip_out_tw_list = [i for i in tw_wip_sites if i not in wip_msa]

    return tw_wip_site_bts_flagged

    # Falta os outros países
tw_wip_site_bts_flagged = check_wip(towerdb, tw_index, tw_wip, tw_bts, msa, tw_index, tw_wip)

print(tw_wip_site_bts_flagged)

Validation DOER sites

In [ ]:
#Validate Decomissioned Sites
def check_decommissioned(df,df_index,status_col, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]

    return filtered[[df_index,status_col, decom_col, doer_col]]


df_decom_errors = check_decommissioned(towerdb,tw_index, tw_status, tw_decom, tw_doer)
df_decom_errors



No errors Founded!



DOER Valitadions

In [ ]:
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    df_tw = df_tw[df_tw[status_col]==status][[tw_index, status_col, date_col]] 
    #print(df_tw)
    if not df_tw[date_col].empty:
        if t == 'doer':
            filtered = df_tw[df_tw[date_col].astype('datetime64[ns]') < current_date]
            return filtered
        else:
            #bill_dates = pd.to_datetime(df_tw[date_col], errors==coerrce)
            filtered = df_tw[df_tw[date_col]=='']
            return filtered
    else:
        print('\nNo errors founded!\n')
actives  = towerdb[towerdb[tw_status]=='In Service']
#Validate sites in service which has DOER less than current date

df_doer_errors = check_tw_bill_doer(actives, tw_index, tw_doer, tw_status, 'In Service', 'doer')
df_doer_errors


TowerDb vs UIS validations

UIS In Month

In [ ]:
"""Check UIP In Month ites matches with Towerdb sites"""
def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col]]
    #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [str(i) for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]

    uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS In Month not active in TowerDB!'])
    #uis_sites_not_in_towerdb['UIS In Month not active in TowerDB!'] =  uis_sites_not_in_towerdb['UIS In Month not active in TowerDB!'].astype(int)
    uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS In Month not active in TowerDB!',\
                                    right_on=tw_index)
    uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS In Month not active in TowerDB!', status_tw_col]]
    
    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS In Month!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS In Month!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS In Month!', status_tw_col]]
    #check for decomissioned site not in uip files

    tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
    decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
    decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                    right_on=tw_index)
    decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]

    #Check BTS sites
    bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
    uip_bts = [i for i in df_uip[df_uip['BTS site applicable charge (Annual)']!='']['Site_ID']]

    #if not set(bts_sites).intersection(uip_sites):
    bts_sites_out_uip = [i for i in uip_bts if i not in bts_sites]
    bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['UIS BTS not in TowerDB(BTS)'])
    #bts_sites_out_uip['UIS BTS not in TowerDB(BTS)'] = bts_sites_out_uip['UIS BTS not in TowerDB(BTS)'].astype(int)
    bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='UIS BTS not in TowerDB(BTS)',\
                                    right_on=tw_index)
    bts_sites_out_uip = bts_sites_out_uip[['UIS BTS not in TowerDB(BTS)', status_tw_col]]

    #  Check for UIP critical sites 
    uip_critical = [i for i in df_uip[df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='']['Site_ID']]
    bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]

    critical = [i for i in uip_critical if i not in bts_tw_critical]
    critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
    critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                right_on=tw_index)
    critical = critical[['Sites with critical level beyond 10% in out UIS File', status_tw_col]]
    #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
    return uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical

uis_sites_not_in_towerdb,in_service_uis_sites, decomiss_sites_in_uis, bts_sites_out_uis, critical = check_uip_tw(towerdb,tw_index, tw_status, \
                                                                tw_decom, tw_bts,tw_critical, uip, uip_sites)

print(uis_sites_not_in_towerdb)
print('\n')
print(in_service_uis_sites)
print('\n')
print(decomiss_sites_in_uis)
print('\n')
print(bts_sites_out_uis)
print('\n')
print(critical)



UIS True UP

In [ ]:
"""Check UIP In Month ites matches with Towerdb sites"""
def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col]]
    #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [str(i) for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]

    uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS In Month not active in TowerDB!'])
    #uis_sites_not_in_towerdb['UIS In Month not active in TowerDB!'] =  uis_sites_not_in_towerdb['UIS In Month not active in TowerDB!'].astype(int)
    uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS In Month not active in TowerDB!',\
                                    right_on=tw_index)
    uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS In Month not active in TowerDB!', status_tw_col]]
    
    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS In Month!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS In Month!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS In Month!', status_tw_col]]
    #check for decomissioned site not in uip files

    tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
    decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
    decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                    right_on=tw_index)
    decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]

    #Check BTS sites
    bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
    uip_bts = [i for i in df_uip[df_uip['BTS site applicable charge (Annual)']!='']['Site_ID']]

    #if not set(bts_sites).intersection(uip_sites):
    bts_sites_out_uip = [i for i in uip_bts if i not in bts_sites]
    bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['UIS BTS not in TowerDB(BTS)'])
    #bts_sites_out_uip['UIS BTS not in TowerDB(BTS)'] = bts_sites_out_uip['UIS BTS not in TowerDB(BTS)'].astype(int)
    bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='UIS BTS not in TowerDB(BTS)',\
                                    right_on=tw_index)
    bts_sites_out_uip = bts_sites_out_uip[['UIS BTS not in TowerDB(BTS)', status_tw_col]]

    #  Check for UIP critical sites 
    uip_critical = [i for i in df_uip[df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='']['Site_ID']]
    bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]

    critical = [i for i in uip_critical if i not in bts_tw_critical]
    critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
    critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                right_on=tw_index)
    critical = critical[['Sites with critical level beyond 10% in out UIS File', status_tw_col]]
    #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
    return uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical

#Read UIP File
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', 'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip_true = pd.read_excel('/content/UserInput_CzechRepublic_20210630 TrueUp.xlsx',sheet_name='SiteLevel',usecols=[0,2,3],skiprows=3).fillna('')
uip_true.columns = uip_names

uip_true_sites = [str(i) for i in uip['Site_ID']]

uis_sites_not_in_towerdb_tu , in_service_uis_sites_tu, decomiss_sites_in_uis_tu, bts_sites_out_uis_tu, critical_tu = check_uip_tw(towerdb,tw_index, tw_status, \
                                                                tw_decom, tw_bts,tw_critical, uip_true, uip_true_sites)

print(uis_sites_not_in_towerdb_tu)
print('\n')
print(in_service_uis_sites_tu)
print('\n')
print(decomiss_sites_in_uis_tu)
print('\n')
print(bts_sites_out_uis_tu)
print('\n')
print(critical_tu)

Commercial Checks

In [ ]:
def check_diffs_v2(path_current, path_last, sheet='Commercial'):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def highlight_diff(data, color='yellow'):
        attr = 'background-color: {}'.format(color)
        other = data.xs('Current', axis='columns', level=-1)
        return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                            index=data.index, columns=data.columns)

    _actual = pd.read_excel(path_current,sheet_name=sheet).fillna('')
    _before = pd.read_excel(path_last,sheet_name=sheet).fillna('')

    df_all = pd.concat([_actual, _before],axis='columns', keys=['Current', 'Last'])
    df_final = df_all.swaplevel(axis='columns')[_actual.columns[1:]]

    return df_final[(_actual != _before).any(1)].style.apply(highlight_diff, axis=None)

path_current = '/content/UserInput_CzechRepublic_20210831.xlsx'
path_last = '/content/UserInput_CzechRepublic_20210731.xlsx'
df_comm_errors = check_diffs_v2(path_current, path_last,sheet='Commercial')
df_comm_errors
#No errors

In [ ]:
def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan']
        #lista = []

        df.fillna("", inplace=True)
        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista

        return df

    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    #df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    if not df_commercial_diffs.empty:
        return df_commercial_diffs
    else:
        print('\nNo Errors founded!')

names = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', 'MSA Sites (Phase 1)\nInput_Value',\
        'PMA Sites (Phase 2)\nInput_Value','Description/Instruction', 'Frequency of Update']
merge_cols = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', \
              'Description/Instruction', 'Frequency of Update']
cols_ordered = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2','Data_Type','Input_Value_1_actual',\
                'Input_Value_1_before','Input_Value_2_actual','Input_Value_2_before','Equal Values','Description/Instruction', 'Frequency of Update']
# Check for commercial Values into current UIP File and compare with UIP File before

df_com = check_commercial(path_current, path_last,['MSA Sites (Phase 1)\nInput_Value','PMA Sites (Phase 2)\nInput_Value'], names, merge_cols, cols_ordered)
df_com
# No Errors


No Errors founded!


In [ ]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
        except AttributeError:
            pass
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
                
    writer.save() 

logs = [['Actives Dates Columns Errors', df_dates_errors],
['General Picklist Errors', df_pick_general_errors],
['Active Picklist Errors', df_pick_actives_errors],
['MSA BTS not in TowerDB', df_mom_bts],
['New Sites',  new_sites],
['BTS Billing trigger Date Errors', df_bts_demerged],
['BTS and WIP Flagged Errors',tw_wip_site_bts_flagged],
['Decomm Sites Dates Errors', df_decom_errors],
['DOER Errors', df_doer_errors],
['UIS Sites not active in TowerDB', uis_sites_not_in_towerdb],
['TowerDB Sites out of UIS', in_service_uis_sites],
['Decom Sites In UIS', decomiss_sites_in_uis],
['UIS BTS not in TowerDB(BTS)', bts_sites_out_uis],
['Sites Beyond 10% out UIS', critical],
['UIS(True Up) not active in TowerDB', uis_sites_not_in_towerdb_tu],
['TowerDB\'s out of UIS(True up)', in_service_uis_sites_tu],
['Decom\'s In UIS(True Up)', decomiss_sites_in_uis_tu],
['UIS(True up) not in TowerDB(BTS)', bts_sites_out_uis_tu],
['Sites Beyond 10% out UIS(True)', critical_tu],
['Comercial Differences', df_comm_errors]]

general_log_erros(logs, '/content/Towerdb_CZ_Errors.xlsx')

Create CSV File

Script to make TA Input file validation

In [4]:
#Read TA Input File
def read_files(path, sheetname, col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int, cols_amount, bill_cols, \
               format='mix', type_date="%d/%m/%Y"):
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df
        
    def replace_values_am(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df 

    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    df = pd.read_excel(path, sheet_name = sheetname,header=0, names = col_name, engine='openpyxl', skiprows = n_skiprows)
    df = df.iloc[:,n_skip_columns:]
    df = df.dropna(subset=[site_index], axis=0)
    df = replace_values_am(df, cols_amount, 0)
    for col in cols_amount:
        #df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int).apply(lambda x: f' {x:,} ')

    for col in cols_int:
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int)
    df = df.fillna('')
    replace_values_am(df, bill_cols)
    date_parser(df, cols_date, format, type_date)
    # define the entiry columns which doesn't have NaN 
    return df

def replace_values_ta(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ']
    #lista = []
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        df[column] = df[column].fillna(0)
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

path_ta_input = '/content/TA_Input_CzechRepublic_20210731.csv'
col = ['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type',\
       'Contract start date', 'Contract end date', 'Status of contract',\
       'Renewal Option', 'Comments', 'Counterpart ID', 'Counterpart',\
       'Classification of Tenant', 'Annual amount - billing currency',\
       'Billing Currency', 'Annual Amount in CZK', 'Amount in EUR',\
       'Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index',\
       'Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key',\
       'Classification', 'Residual Period in Years', 'Remarks',\
       'Count of unique key', 'Count of contracts', 'Scoping classification',\
       'DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month',\
       'Updated Item', 'Comment', 'Counterpart ID_1', 'Counterpart_1', 'x',\
       'SiteType', '1.028', 'Unique Tenant', 'Unique Tenant Count','Quota of Unique Key','unamed']

#ta = pd.read_csv(path_ta_input, header=0, names=col)
dates = ['Contract start date', 'Contract end date', 'Update in month']
interger = ['Residual Period in Years','Count of unique key', 'Count of contracts', 'Unique Tenant Count',\
            'Quota of Unique Key']
amount = ['Annual Amount in CZK', 'Amount in EUR']
ta_bill_cols = ['Contract start date','Contract end date',\
                'Counterpart','Classification of Tenant']
ta = read_files('/content/TowerDB_FY21-06 June-Skylon VF CZ_v1.xlsx', 'Tenant ',col, 0, 0, 'Code', dates, interger,amount, ta_bill_cols)

ta_cols = ['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type','Contract start date', \
           'Contract end date', 'Status of contract','Renewal Option', 'Comments', 'Counterpart ID', \
           'Counterpart','Classification of Tenant', 'Annual amount - billing currency','Billing Currency', \
           'Annual Amount in CZK', 'Amount in EUR','Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index',\
           'Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key','Classification', \
           'Residual Period in Years', 'Remarks','Count of unique key', 'Count of contracts', \
           'Scoping classification','DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month',\
           'Updated Item', 'Comment', 'Counterpart ID_1', 'Counterpart_1', 'x','SiteType',\
           '1.028', 'Unique Tenant', 'Unique Tenant Count', 'unamed', 'unamed_2']

ta = ta.reindex(columns=ta_cols)
ta = ta.rename(columns={'unamed': '',
                        'unamed_2': ''})

#ta.to_excel('/content/TA_Input_CzechRepublic_20210831.xlsx', index=False)
ta.to_csv('/content/TA_Input_CzechRepublic_20210831.csv', index=False)

In [ ]:
ta_cols = ['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type','Contract start date', \
           'Contract end date', 'Status of contract','Renewal Option', 'Comments', 'Counterpart ID', \
           'Counterpart','Classification of Tenant', 'Annual amount - billing currency','Billing Currency', \
           'Annual Amount in CZK', 'Amount in EUR','Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index',\
           'Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key','Classification', \
           'Residual Period in Years', 'Remarks','Count of unique key', 'Count of contracts', \
           'Scoping classification','DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month',\
           'Updated Item', 'Comment', 'Counterpart ID_1', 'Counterpart_1', 'x','SiteType',\
           '1.028', 'Unique Tenant', 'Unique Tenant Count', 'unamed', 'unamed_2']
len(ta_cols)

45

In [ ]:
ta_msa = pd.read_csv('/content/TA_Input_CzechRepublic_20210731.csv')
msa_cols = list(ta_msa.columns)
ta_cols = list(ta.columns)
df_cols = check_columns_received(ta, msa_cols)
df_cols

In [ ]:
def check_lc_ta_dates(df,tw_index,start_date,end_date):
    df[start_date] = pd.to_datetime(df[start_date],errors='coerce')
    df[end_date] = pd.to_datetime(df[end_date],errors='coerce')
    filtered = df[df[start_date] > df[end_date]]
    if not filtered.empty:
        return filtered[[tw_index, start_date,end_date]]
    else:
        print('\nNo Errors Founded!\n')

df_ta_dates = check_lc_ta_dates(ta,'Code', 'Contract start date', 'Contract end date')
df_ta_dates


No Errors Founded!



In [ ]:
ta['Classification of Tenant'].value_counts()

MNO     323
OTMO    295
Name: Classification of Tenant, dtype: int64

Script to read LC Input file


In [ ]:
def read_files(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int, cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y"):
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df
    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    df = pd.read_excel(path, sheet_name = sheetname,header=0, names = col_name, engine='openpyxl', skiprows = n_skiprows)
    df = df.iloc[:,n_skip_columns:]
    df = df.dropna(subset=[site_index], axis=0)
    df = replace_values(df, cols_amount, 0)
    for col in cols_amount:
        #df[col] = df[col].fillna(0)
        df[col] = df[col].apply(lambda x: '{:0,.2f}'.format(x))

    df = df.fillna('')
    replace_values(df, bill_cols)
    date_parser(df, cols_date, format, type_date)
    # define the entiry columns which doesn't have NaN 
    return df

def replace_values_ta(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', ]
    #lista = []
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        df[column] = df[column].fillna(0)
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

col_names = ['Contract ID - NEW', 'FIN ID', 'Contract Crncy', 'Contract Type', 'Freq.', 'Freq. Unit', 'Indexation',\
         'Index upon request', 'Counterpart ID', 'Counterpart', 'LC Amount CZK\nyearly', 'Amount in EUR yearly (Actual)', \
         'Payment terms Code', 'VAT Subject', '% of VAT', 'Contr. Start date', 'Contr. 1st End', 'Contr. Term End', \
         'Sublease Consession', 'Renewal Option', 'Unique Key', 'Count of Key', 'Count of ContrBct ', \
         'Remarks - Consistency check', 'Residual Period in years', 'Scoping classification', 'DAS sites', \
         'DAS ownership', '31_12_9999', 'Unique Key1', 'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021', \
         'Check LC amount', 'Updated Item\nAmount yearly', 'Contr. 1st End as of 31.05.2021', \
         'Contr. 1st End as of 30.06.2021', 'Check Contr. 1st End', 'Updated Item\n1st End date', 'Comment', \
         'Comment date', 'Date']

#ta = pd.read_csv(path_ta_input, header=0, names=col)
dates_lc = ['Contr. Start date', 'Contr. 1st End', 'Contr. 1st End as of 31.05.2021', 'Contr. 1st End as of 30.06.2021']
interger_lc = []
amount_lc = ["Amount in EUR yearly (Actual)", 'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021']
lc_bill_cols = ['Contract ID - NEW','Counterpart']
lc = read_files('/content/TowerDB_FY21-06 June-Skylon VF CZ_v1.xlsx', 'Final data_Lease',col_names, 1, 0, 'FIN ID', \
                dates_lc, interger_lc,amount_lc, lc_bill_cols)

col_order_lc = ["Contract ID - NEW","FIN ID","Contract Crncy","Contract Type","Freq.","Freq. Unit","Indexation",\
                "Index upon request","Counterpart ID","Counterpart","LC Amount CZK\nyearly","Amount in EUR yearly (Actual)",\
                "Payment terms Code","VAT Subject","% of VAT","Contr. Start date","Contr. 1st End","Contr. Term End",\
                "Sublease Consession","Renewal Option","Unique Key","Count of Key","Count of ContrBct ",\
                "Remarks - Consistency check","Residual Period in years","Scoping classification","DAS sites",\
                "DAS ownership","31_12_9999","Unique Key1",'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021',\
                "Check LC amount","Updated Item\nAmount yearly",'Contr. 1st End as of 31.05.2021','Contr. 1st End as of 30.06.2021',\
                "Check Contr. 1st End","Updated Item\n1st End date","Comment","Comment date","Date"]
lc = lc[col_order_lc]
lc.head(2)

In [ ]:
lc.to_excel('/content/LC_Input_CzechRepublic_20210831.xlsx', index=False)

In [ ]:
lc.to_csv('/content/LC_Input_CzechRepublic_20210831.csv', index=False)

In [ ]:
def change_incorret(df, df_index, col_chg, incorrect_value, new_value):
    list_sites = list(df[df[col_chg]==incorrect_value][df_index])
    df.loc[df[df_index].isin(list_sites), col_chg] = new_value
    print(df.loc[df[df_index].isin(list_sites)][[df_index, col_chg]])

change_incorret(lc, "Contract ID - NEW", '% of VAT', 0.21,'21%')


In [ ]:
def check_amounts_lc(df_check, df_index, columns, pattern=','):

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)
    
    filtered = df_check[[df_index]]

    for site in df[df_index]:
        for column in columns[1:]:
            if not str(df.loc[site,column]).__contains__(pattern):
                #print(df.loc[site,column])
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                else:
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    if not df_new.empty:
        #df_new = pd.merge(df_new, filtered, how='left', left_on='identification - site key', right_on=tw_index)
        df_new = df_new.set_index(df_index)
        #df_new = df_new[[status_col]+ df_new.columns[:-1].tolist()]
        #df_new = df_new[['identification - site key', status_col]]
        return df_new
    else: 
        print('\nNo one columns with incorrect Amount format!\n') 

lc_cols = ["FIN ID", 'LC Amount CZK\nyearly']  

df_lc_amount = check_amounts_lc(lc, "FIN ID", lc_cols)
df_lc_amount
# No errors

In [ ]:
def check_lc_ta_dates(df,tw_index,start_date,end_date):
    df[start_date] = pd.to_datetime(df[start_date],errors='coerce')
    df[end_date] = pd.to_datetime(df[end_date],errors='coerce')
    filtered = df[df[start_date] > df[end_date]]
    if not filtered.empty:
        return filtered[[tw_index, start_date,end_date]]
    else:
        print('\nNo Errors Founded!\n')
        
df_lc_dates = check_lc_ta_dates(lc,"FIN ID", "Contr. Start date","Contr. 1st End")
df_lc_dates



No Errors Founded!

